In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-10-31


Found rundate 2023-10-30 10:00:02 < 2023-10-31 at commit 69aa2952; returning commit bd9d9d1


'2023-10-31 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-10-31 10:00:02, NJSP was reporting 493 YTD deaths
Current YTD Deaths (2024-10-31 10:00:03-04:00): 550
Previous year YTD Deaths (adjusted; 2023-10-31 10:00:02): 493.2736355703835
Projected 2024 total: 673.5071520378281
11.5% change


FAUQStats cache miss: 2023, Tue Oct 31 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

CCODE      CNAME MCODE            MNAME HIGHWAY  \
ACCID                                                   
12249    07      Essex  0714      Newark City     NaN   
12251    01   Atlantic  0112    Hamilton Twsp     322   
12253    12  Middlesex  1217  Piscataway Twsp     NaN   
12254    02     Bergen  0221    Garfield City     NaN   
12267    07      Essex  0714      Newark City     NaN   
...     ...        ...   ...              ...     ...   
12776    12  Middlesex  1219  Sayreville Boro      35   
12773    13   Monmouth  1304  Colts Neck Twsp     537   
12782    07      Essex  0713   Montclair Town     NaN   
12783    07      Essex  0709   Irvington Town     NaN   
12775    04     Camden  0436     Winslow Twsp     446   

                               LOCATION  FATALITIES  FATAL_D  FATAL_P  \
ACCID                                                                   
12249                       Mulberry St         1.0      0.0      0.0   
12251         State Highway 322 MP 46.4         1.0      1.0      0.0   
12253          South Randolphville Road         1.0      1.0      0.0   
12254                       Outwater Ln         1.0      0.0      0.0   
12267                     Washington St         1.0      0.0      0.0   
...                                 ...         ...      ...      ...   
12776                  State Highway 35         1.0      0.0      1.0   
12773                        County 537         1.0      1.0      0.0   
12782                      Bellevue Ave         1.0      0.0      0.0   
12783                       Clinton Ave         1.0      0.0      0.0   
12775  Atlantic City Expressway MP 37.6         1.0      1.0      0.0   

       FATAL_T  FATAL_B                    STREET  INJURIES  \
ACCID                                                         
12249      1.0      0.0               Mulberry St       NaN   
12251      0.0      0.0                       NaN       0.0   
12253      0.0      0.0  South Randolphville Road       0.0   
12254      1.0      0.0               Outwater Ln       0.0   
12267      1.0      0.0             Washington St       NaN   
...        ...      ...                       ...       ...   
12776      0.0      0.0                       NaN       NaN   
12773      0.0      0.0                       NaN       NaN   
12782      1.0      0.0              Bellevue Ave       NaN   
12783      1.0      0.0               Clinton Ave       0.0   
12775      0.0      0.0                       NaN       NaN   

                             dt  
ACCID                            
12249 2023-01-01 01:49:00-05:00  
12251 2023-01-02 09:26:00-05:00  
12253 2023-01-02 19:41:00-05:00  
12254 2023-01-03 20:52:00-05:00  
12267 2023-01-04 06:21:00-05:00  
...                         ...  
12776 2023-10-28 05:00:00-04:00  
12773 2023-10-28 05:50:00-04:00  
12782 2023-10-28 12:15:00-04:00  
12783 2023-10-28 22:20:00-04:00  
12775 2023-10-29 06:22:00-04:00  

[468 rows x 14 columns]

Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

driver  passenger  pedestrian  cyclist  crashes
county                                                     
Atlantic        26          2          15        3       45
Bergen          14          4          15        2       33
Burlington      26          4           6        1       37
Camden          14          3          13        0       30
Cape May         6          0           0        0        6
Cumberland      16          6           5        0       22
Essex           17          5          21        0       42
Gloucester      15          4           3        0       20
Hudson           7          2           4        0       12
Hunterdon        4          2           2        0        8
Mercer          13          5           8        0       25
Middlesex       23         10          18        1       46
Monmouth        19          5          14        4       41
Morris           7          1           5        1       14
Ocean           25          4          11        1       39
Passaic         16          2           8        0       25
Salem            7          3           1        3       12
Somerset        11          0           4        0       15
Sussex           6          2           2        1       11
Union           13          4          10        1       25
Warren          10          1           2        1       13

Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

83.2% through the year, 16.8% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                              
Atlantic   driver        15       18       26
Bergen     driver        18       20       14
Burlington driver        18       26       26
Camden     driver        17       18       14
Cape May   driver         4        4        6
...                     ...      ...      ...
Salem      crashes        9       11       12
Somerset   crashes       17       21       15
Sussex     crashes        4        6       11
Union      crashes       22       35       25
Warren     crashes        9       11       13

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type        crashes  cyclist  driver  passenger  pedestrian
county                                                     
Atlantic         57        6      31          2          19
Bergen           39        2      16          4          21
Burlington       52        1      37          5           7
Camden           39        0      15          4          23
Cape May          8        0       6          0           0
Cumberland       26        0      17          7           7
Essex            55        0      20          8          30
Gloucester       24        0      17          6           5
Hudson           13        0       8          2           5
Hunterdon        10        0       6          2           2
Mercer           32        0      15          5          11
Middlesex        49        1      24         15          18
Monmouth         46        4      21          5          18
Morris           17        1       8          2           5
Ocean            42        1      26          5          12
Passaic          31        0      20          4           9
Salem            15        4       7          3           2
Somerset         19        0      13          0           6
Sussex           16        1       9          2           2
Union            39        1      20          6          18
Warren           16        1      14          1           2

In [9]:
projected.to_csv(PROJECTED_CSV)